# Setting Up:

In [ ]:
from google.colab import drive
drive.mount("content")

Mounted at content


In [ ]:
!pip install -q derivative
!pip install -q gurobipy
!pip install -q pysindy==1.7.3
!pip install -q numpy==1.26.4
!pip install -q scipy==1.11.4

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 60.4/60.4 kB 2.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 35.8/35.8 MB 25.1 MB/s eta 0:00:00
  Attempting uninstall: scipy
    Found existing installation: scipy 1.16.1
    Uninstalling scipy-1.16.1:
      Successfully uninstalled scipy-1.16.1
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
tsfresh 0.21.1 requires scipy>=1.14.0; python_version >= "3.10", but you have scipy 1.11.4 which is incompatible.


In [ ]:
import h5py
import gurobipy
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import pysindy as ps
from scipy.interpolate import RegularGridInterpolator
from tqdm import tqdm

# Getting Data:

In [ ]:
def get_data(data_path, grid_path, x_lims, z_lims, t_lims):
    f1 = h5py.File(f"{data_path}/data.h5",'r')
    f2 = h5py.File(f"{grid_path}/grid.h5",'r')

    # We will not take the full domain, but just a truncated part of the domain
    x1, x2 = x_lims
    z1, z2 = z_lims
    t1, t2 = t_lims

    data, time = f1["data"], f1["time"][t1:t2]
    xdomain, zdomain = f2["x"][x1:x2][:], f2["y"][z1:z2][:]

    Pres = data[x1:x2,z1:z2,t1:t2,0][:]
    Temp = data[x1:x2,z1:z2,t1:t2,1][:]
    U =data[x1:x2,z1:z2,t1:t2,2][:]
    W = data[x1:x2,z1:z2,t1:t2,3][:]

    f1.close(), f2.close()

    return xdomain, zdomain, time, Pres, Temp, U, W

In [ ]:
data_path = "/content/content/MyDrive/Learning/Machine Learning for ML Flow/RB_2D"

# Truncating the domain
x1, x2 = 100, 356
z1, z2 = 0, 128
t1, t2 = 0,50

xdomain, zdomain, time, Pres, Temp, U, W = get_data(data_path, data_path, (x1,x2), (z1,z2), (t1,t2))

In [ ]:
Pres.shape

(256, 128, 50)

In [ ]:
xdomain.shape, zdomain.shape, time.shape

((256,), (128,), (50,))

## Interpolating grid

In [ ]:
def interpolate(xdomain, zdomain, time, data):
    nx, nz,nt = xdomain.shape[0], zdomain.shape[0], time.shape[0]

    xdomain_uniform = np.linspace(xdomain.min(), xdomain.max(), nx)
    zdomain_uniform = np.linspace(zdomain.min(), zdomain.max(), nz)
    new_x_grid, new_z_grid = np.meshgrid(xdomain_uniform, zdomain_uniform, indexing='ij')  # Create the grid of points where we want to evaluate the data
    evaluation_points = np.stack([new_x_grid.ravel(), new_z_grid.ravel()], axis=-1)   # We need to stack these into a list of (x, z) points for the interpolator

    # Preallocating matrices
    data_uniform = np.zeros((nx, nz, nt))

    for t in tqdm(range(nt)):
        # Get the 2D spatial slice for the current time step
        current_slice = Pres[:, :, t]

        # Create the interpolation function for this slice
        # This function "learns" the data based on the non-uniform grid
        interp_func = RegularGridInterpolator(
            (xdomain, zdomain), # Original grid points
            current_slice,                          # Original data values
            method='linear'
        )

        # Evaluate the function on our new grid of points
        interpolated_slice_flat = interp_func(evaluation_points)

        # The result is a flat 1D array, so we reshape it back to our 2D grid shape
        data_uniform[:, :, t] = interpolated_slice_flat.reshape((nx, nz))

    return data_uniform


In [ ]:
# Interpolate the variables in uniform grid
Pres = interpolate(xdomain, zdomain, time, Pres)
Temp = interpolate(xdomain, zdomain, time, Temp)
U = interpolate(xdomain, zdomain, time, U)
W = interpolate(xdomain, zdomain, time, W)

# Also get the uniform grid
nx, nz,nt = xdomain.shape[0], zdomain.shape[0], time.shape[0]

xdomain = np.linspace(xdomain.min(), xdomain.max(), nx)
zdomain = np.linspace(zdomain.min(), zdomain.max(), nz)


100%|██████████| 50/50 [00:00<00:00, 265.07it/s]


# PySINDy

In [ ]:
spatiotemporal_grid = np.zeros((nx, nz,  nt, 3))

spatiotemporal_grid[:,:,:,0] = xdomain[:,np.newaxis, np.newaxis]
spatiotemporal_grid[:, :,:,1] = zdomain[np.newaxis, :, np.newaxis]
spatiotemporal_grid[:, :,:, 2] = time[np.newaxis, np.newaxis, :]

# X, T = np.meshgrid(zdomain, time)
XT = spatiotemporal_grid
XT.shape

(256, 128, 50, 3)

## WeakPDE Setup

**`ps.WeakPDELibrary`**: This is the heart of the discovery process. Instead of calculating derivatives by finite differences (which can be very noisy), the "weak form" library works with integrals over small subdomains of your data. This is much more robust to noise.
    
-   `derivative_order=2`: It will automatically compute all spatial derivatives up to the second order (e.g., U\_x,U\_z,U\_xx,U\_zz,U\_xz).
    
-   `include_interaction=True`: This is critical for finding nonlinear PDEs. It will create products of all the terms generated so far. For example, it combines U and U\_x to create the advection term U⋅U\_x.
    
-   `K`: The number of random subdomains to integrate over. Think of this as an ensembling method to get a more stable estimate.
    
-   `H_xt`: The size of these small subdomains in the x, z, and time directions.

In [ ]:
# Subdomain divisions
divisions_x = 48
divisions_z = 10
divisions_t = 8

K = 300  # nu of subdomains

In [ ]:
weak_lin_lib = ps.WeakPDELibrary(
    library_functions=[lambda x: x],
    function_names = [lambda x: x],
    derivative_order=2,
    spatiotemporal_grid=XT,
    include_interaction = True,
    periodic = False,
    p=6, # Our highest order term is second derivatives so we need to make sure this is set right
    #uniform=False, there's some argument we should check is false
    K=K, #number of sub-domains to evaluate the integral on, essentially an ensembling process
    H_xt=[(xdomain[-1]-xdomain[0]) / divisions_x, (zdomain[-1]-zdomain[0]) / divisions_z, (time[-1]-time[0]) / divisions_t]
)

## Applying Constraints

**Defining Constraints**: The script defines several types of constraints:

1.  **Removing Terms**: `(equation_idx, term_name, None, None)` sets the coefficient for `term_name` in that specific equation to zero. The code does this to remove nonlinear diffusive terms (like (U2)\_xx) which are not present in the Navier-Stokes equations.
    
2.  **Incompressibility**: The code attempts to enforce the incompressibility condition (∇⋅u\=∂x∂U+∂z∂W\=0). By setting terms like U∂z∂W equal to −U∂x∂U, it simplifies the advection term U∂x∂U+W∂z∂U.
    
3.  **Symmetry/Equality Constraints**: `(eqn1, term1, eqn2, term2)` forces the coefficient of `term1` in `eqn1` to be equal to the coefficient of `term2` in `eqn2`. For example, it enforces that the diffusion coefficient is the same for U\_xx and U\_zz (isotropic viscosity).

In [ ]:
lib_terms = ['U', 'W', 'P', 'T', 'U_2', 'W_2', 'P_2', 'T_2', 'U_22', 'W_22', 'P_22', 'T_22',
'U_1', 'W_1', 'P_1', 'T_1', 'U_12', 'W_12', 'P_12', 'T_12', 'U_11', 'W_11', 'P_11', 'T_11',
'UU_2', 'WU_2', 'PU_2',
'TU_2', 'UW_2', 'WW_2', 'PW_2', 'TW_2', 'UP_2', 'WP_2', 'PP_2', 'TP_2', 'UT_2', 'WT_2',
'PT_2', 'TT_2', 'UU_22', 'WU_22',
'PU_22', 'TU_22', 'UW_22', 'WW_22', 'PW_22', 'TW_22', 'UP_22', 'WP_22', 'PP_22', 'TP_22',
'UT_22', 'WT_22', 'PT_22', 'TT_22',
'UU_1', 'WU_1', 'PU_1', 'TU_1', 'UW_1', 'WW_1', 'PW_1', 'TW_1', 'UP_1', 'WP_1', 'PP_1',
'TP_1', 'UT_1', 'WT_1', 'PT_1', 'TT_1',
'UU_12', 'WU_12', 'PU_12', 'TU_12', 'UW_12', 'WW_12', 'PW_12', 'TW_12', 'UP_12', 'WP_12',
'PP_12', 'TP_12', 'UT_12', 'WT_12',
'PT_12', 'TT_12', 'UU_11', 'WU_11', 'PU_11', 'TU_11', 'UW_11', 'WW_11', 'PW_11', 'TW_11',
'UP_11', 'WP_11', 'PP_11', 'TP_11',
 'UT_11', 'WT_11', 'PT_11', 'TT_11']
lib_terms = np.array(lib_terms)

In [ ]:
# The constraints have to be of the size (n_constraints, n_features * n_targets)

def constrain_matrix(constraint_tuples):
    """
    specify a list of tuples [(eqn1, term1, eqn2, term2)]
    where eqn1 is the equation you want to constrain, and term1 is the term you want to constraint.

    if you specify (eqn1, term1, None, None), then term1 in equation 1 is automatically set to zero.
    If you specifiy (eqn1, term1, eqn2, term2), then term1 is constrained to equal term2 in eqn2
    """
    n_features = len(lib_terms)
    n_eqns = 4
    n_constraints = len(constraint_tuples)
    constraintlhs =np.zeros((n_constraints, int(n_features*n_eqns)))
    constraintrhs = np.zeros(n_constraints)  # every term on rhs has to be 0, constraintlhs @ features = constraintrhs

    #for each constraint
    for ii, tup in enumerate(constraint_tuples):
        term_index = np.where(lib_terms == tup[1])[0]
        constraintlhs[ii, term_index + n_features*tup[0]] = 1 #set the term we want to constrain, in the equation

        if tup[-1] is None:
            pass
        else:
            term_index = np.where(lib_terms == tup[-1])[0]
            constraintlhs[ii, term_index+ n_features*tup[2]] = -1 #constrain coefficients to be equal

    return constraintlhs, constraintrhs



In [ ]:
# Remove all diffusive terms except U_11, U_22, W_11, W_22, T_11, T_22 i.e. remove 54 of 60.
nonlinear_diffusive = ['P_22', 'P_12', 'T_12', 'P_11', 'UU_22','WU_22','PU_22','TU_22',
                       'UW_22','WW_22','PW_22','TW_22','UP_22','WP_22','PP_22','TP_22','UT_22','WT_22',
                       'PT_22','TT_22',
                       'UP_12','WP_12','PP_12','TP_12','UT_12','WT_12','PT_12','TT_12','UU_11','WU_11',
                       'PU_11','TU_11','UW_11','WW_11','PW_11','TW_11','UP_11','WP_11','PP_11','TP_11',
                       'UT_11','WT_11','PT_11','TT_11']
constraint_tuples = [(j, elem, None, None) for j in [0,1,3] for elem in nonlinear_diffusive]

In [ ]:
# Constraint from incompressibility: Grad U = 0 (U_1=-W_2)
IncomConstr_Ueqn =[(0, 'W_2', None, None), (0, 'UW_2', None, None), (0, 'WW_2', None, None),
                                           (0, 'PW_2', None, None), (0, 'TW_2', None, None),
                   (0,'W_12', None, None), (0,'UW_12', None, None), (0,'WW_12', None, None),
                                           (0,'PW_12', None, None), (0,'TW_12', None, None),
                   (0,'U_12', None, None), (0,'UU_12', None, None), (0,'WU_12', None, None),
                                           (0,'PU_12', None, None), (0,'TU_12', None, None)]
constraint_tuples += IncomConstr_Ueqn
IncomConstr_Weqn =[(1, 'U_1', None, None), (1, 'UU_1', None, None), (1, 'WU_1', None, None),
                                           (1, 'PU_1', None, None), (1, 'TU_1', None, None),
                   (1,'U_12', None, None), (1,'UU_12', None, None), (1,'WU_12', None, None),
                                           (1,'PU_12', None, None), (1,'TU_12', None, None),
                   (1,'W_12', None, None), (1,'UW_12', None, None), (1,'WW_12', None, None),
                                           (1,'PW_12', None, None), (1,'TW_12', None, None)]
constraint_tuples += IncomConstr_Weqn

In [ ]:
get_rid_of_p = [(2, elem, None, None) for elem in lib_terms]
constraint_tuples += get_rid_of_p

In [ ]:
constraint_tuples += [(0, "UU_1", 0, "WU_2")] # symmetry constraint
constraint_tuples += [(0, "UU_1", 1, "UW_1")] # vector equation constraint
constraint_tuples += [(0, "UU_1", 1, "WW_2")] # vector equation constraint
constraint_tuples += [(0, "U_11", 0, "U_22")] # symmetry constraint
constraint_tuples += [(0, "U_11", 1, "W_11")] # vector equation constraint
constraint_tuples += [(0, "U_22", 1, "W_22")] # vector equation constraint
constraint_tuples += [(0,  "P_1", 1,  "P_2")] # vector equation constraint
constraint_tuples += [(1, "UW_1", 1, "WW_2")] # symmetry constraint
constraint_tuples += [(1, "W_11", 1, "W_22")] # symmetry constraint
constraint_tuples += [(3, "UT_1", 3, "WT_2")] # symmetry constraint
constraint_tuples += [(3, "T_11", 3, "T_22")] # symmetry constraint
constraint_tuples += [(0, "W_11", None, None)]
constraint_tuples += [(0, "W_22", None, None)]
constraint_tuples += [(0, "T_11", None, None)]
constraint_tuples += [(0, "T_22", None, None)]
constraint_tuples += [(1, "U_11", None, None)]
constraint_tuples += [(1, "U_22", None, None)]
constraint_tuples += [(1, "T_11", None, None)]
constraint_tuples += [(1, "T_22", None, None)]
constraint_tuples += [(3, "U_11", None, None)]
constraint_tuples += [(3, "U_22", None, None)]
constraint_tuples += [(3, "W_11", None, None)]
constraint_tuples += [(3, "W_22", None, None)]

constraintlhs, constraintrhs = constrain_matrix(constraint_tuples)

In [ ]:
constraintlhs.shape, constraintrhs.shape

((289, 416), (289,))

## Figuring out equations

In [ ]:
n_targets = 4 #number of equations
Rayleigh = 1e8
Prandtl = 1
#     Kinematic viscosity
nu = (Rayleigh / Prandtl) ** (-1/2)
#     Thermal diffusivity
kappa = (Rayleigh * Prandtl) ** (-1/2)

In [ ]:
X = np.zeros((U.shape[0], U.shape[1], U.shape[2], 4))
X[:,:,:,0] = U
X[:,:,:,1] = W
X[:,:,:,2] = Pres
X[:,:,:,3] = Temp

time -=  time[0]  # make time start from 0

In [ ]:
X_dot_train_integral = weak_lin_lib.convert_u_dot_integral(X)

In [ ]:
X.shape

(256, 128, 50, 4)

## Optimizer

Gurobi requires license, then store it in the form of



```
{"LICENSEID":ID,
"WLSACCESSID":"WLSACCESSID",
"WLSSECRET":"WLSSECRET"
}
```

in colab's secret key


In [ ]:
from google.colab import userdata
gurobi_license = userdata.get('Gurobi')

import json
gurobi_license = json.loads(gurobi_license)

In [ ]:
import gurobipy as gp
import numpy as np
from sklearn.utils.validation import check_is_fitted

class MIOSR(ps.optimizers.BaseOptimizer):
    """Mixed-Integer Optimized Sparse Regression.

    Solves the sparsity constrained regression problem to provable optimality
    .. math::

        \\|y-Xw\\|^2_2 + \\lambda R(u)

    .. math::

        \\text{subject to } \\|w\\|_0 \\leq k

    by using type-1 specially ordered sets (SOS1) to encode the support of
    the coefficients. Can optionally add additional constraints on the
    coefficients or access the gurobi model directly for advanced usage.
    See the following reference for additional details:

        Bertsimas, D. and Gurnee, W., 2022. Learning Sparse Nonlinear Dynamics
        via Mixed-Integer Optimization. arXiv preprint arXiv:2206.00176.

    Parameters
    ----------
    target_sparsity : int, optional (default 5)
        The maximum number of nonzero coefficients across all dimensions.
        If set, the model will fit all dimensions jointly, potentially reducing
        statistical efficiency.

    group_sparsity : int tuple, optional (default None)
        Tuple of length n_targets constraining the number of nonzero
        coefficients for each target dimension.

    alpha : float, optional (default 0.01)
        Optional L2 (ridge) regularization on the weight vector.

    regression_timeout : int, optional (default 10)
        The timeout (in seconds) of the gurobi optimizer to solve and prove
        optimality (either per dimension or jointly depending on the
        above sparsity settings).

    fit_intercept : boolean, optional (default False)
        Whether to calculate the intercept for this model. If set to false, no
        intercept will be used in calculations.

    constraint_lhs : numpy ndarray, optional (default None)
        Shape should be (n_constraints, n_features * n_targets),
        The left hand side matrix C of Cw <= d.
        There should be one row per constraint.

    constraint_rhs : numpy ndarray, shape (n_constraints,), optional (default None)
        The right hand side vector d of Cw <= d.

    constraint_order : string, optional (default "target")
        The format in which the constraints ``constraint_lhs`` were passed.
        Must be one of "target" or "feature".
        "target" indicates that the constraints are grouped by target:
        i.e. the first ``n_features`` columns
        correspond to constraint coefficients on the library features
        for the first target (variable), the next ``n_features`` columns to
        the library features for the second target (variable), and so on.
        "feature" indicates that the constraints are grouped by library
        feature: the first ``n_targets`` columns correspond to the first
        library feature, the next ``n_targets`` columns to the second library
        feature, and so on.

    normalize_columns : boolean, optional (default False)
        Normalize the columns of x (the SINDy library terms) before regression
        by dividing by the L2-norm. Note that the 'normalize' option in sklearn
        is deprecated in sklearn versions >= 1.0 and will be removed. Note that
        this parameter is incompatible with the constraints!

    copy_X : boolean, optional (default True)
        If True, X will be copied; else, it may be overwritten.

    initial_guess : np.ndarray, shape (n_features) or (n_targets, n_features), \
            optional (default None)
        Initial guess for coefficients ``coef_`` to warmstart the optimizer.

    verbose : bool, optional (default False)
        If True, prints out the Gurobi solver log.

    Attributes
    ----------
    coef_ : array, shape (n_features,) or (n_targets, n_features)
        Weight vector(s).
    ind_ : array, shape (n_features,) or (n_targets, n_features)
        Array of 0s and 1s indicating which coefficients of the
        weight vector have not been masked out, i.e. the support of
        ``self.coef_``.
    model : gurobipy.model
        The raw gurobi model being solved.
    """

    def __init__(
        self,
        target_sparsity=5,
        group_sparsity=None,
        alpha=0.01,
        regression_timeout=10,
        fit_intercept=False,
        constraint_lhs=None,
        constraint_rhs=None,
        constraint_order="target",
        normalize_columns=False,
        copy_X=True,
        initial_guess=None,
        verbose=False,
        gurobi_license=None
    ):
        super(MIOSR, self).__init__(
            normalize_columns=normalize_columns,
            fit_intercept=fit_intercept,
            copy_X=copy_X,
        )

        if target_sparsity is not None and (
            target_sparsity <= 0 or not isinstance(target_sparsity, int)
        ):
            raise ValueError("target_sparsity must be positive int")
        if constraint_order not in {"target", "feature"}:
            raise ValueError("constraint_order must be one of {'target', 'feature'}")
        if alpha < 0:
            raise ValueError("alpha cannot be negative")

        self.target_sparsity = target_sparsity
        self.group_sparsity = group_sparsity
        self.constraint_lhs = constraint_lhs
        self.constraint_rhs = constraint_rhs
        self.constraint_order = constraint_order
        self.alpha = alpha
        self.initial_guess = initial_guess
        self.regression_timeout = regression_timeout
        self.verbose = verbose

        # Gurobi License
        self.gurobi_license = gurobi_license
        self.gurobi_license["OutputFlag"] = 0

        self.model = None

    def _make_model(self, X, y, k, warm_start=None):
        # model = gp.Model()
        env = gp.Env(params=self.gurobi_license)

        # 4. Create the model within the licensed environment
        model = gp.Model("wls_example", env=env)



        n_samples, n_features = X.shape
        _, n_targets = y.shape

        coeff_var = model.addMVar(
            n_targets * n_features,
            lb=-gp.GRB.INFINITY,
            vtype=gp.GRB.CONTINUOUS,
            name="coeff_var",
        )
        iszero = model.addMVar(
            n_targets * n_features, vtype=gp.GRB.BINARY, name="iszero"
        )

        # Sparsity constraint
        for i in range(n_targets * n_features):
            model.addSOS(gp.GRB.SOS_TYPE1, [coeff_var[i], iszero[i]])
        model.addConstr(iszero.sum() >= (n_targets * n_features) - k, name="sparsity")

        # Group sparsity constraints
        if self.group_sparsity is not None and n_targets > 1:
            for i in range(n_targets):
                dimension_sparsity = self.group_sparsity[i]
                model.addConstr(
                    iszero[i * n_features : (i + 1) * n_features].sum()
                    >= n_features - dimension_sparsity,
                    name=f"group_sparsity{i}",
                )

        # General equality constraints
        if self.constraint_lhs is not None and self.constraint_rhs is not None:
            if self.constraint_order == "feature":
                target_indexing = (
                    np.arange(n_targets * n_features)
                    .reshape(n_targets, n_features, order="F")
                    .flatten()
                )
                constraint_lhs = self.constraint_lhs[:, target_indexing]
            else:
                constraint_lhs = self.constraint_lhs
            model.addConstr(
                constraint_lhs @ coeff_var == self.constraint_rhs, name="coeff_constrs"
            )

        if warm_start is not None:
            warm_start = warm_start.reshape(1, n_targets * n_features)[0]
            for i in range(n_features):
                iszero[i].start = abs(warm_start[i]) < 1e-6
                coeff_var[i].start = warm_start[i]

        # Equation 15 in paper
        Quad = np.dot(X.T, X)
        obj = self.alpha * (coeff_var @ coeff_var)
        for i in range(n_targets):
            lin = np.dot(y[:, i].T, X)
            obj += (
                coeff_var[n_features * i : n_features * (i + 1)]
                @ Quad
                @ coeff_var[n_features * i : n_features * (i + 1)]
            )
            obj -= 2 * (lin @ coeff_var[n_features * i : n_features * (i + 1)])

        model.setObjective(obj, gp.GRB.MINIMIZE)

        model.params.OutputFlag = 1 if self.verbose else 0
        model.params.timelimit = self.regression_timeout
        model.update()

        self.model = model

        return model, coeff_var

    def _regress(self, X, y, k, warm_start=None):
        """
        Deploy and optimize the MIO formulation of L0-Regression.
        """
        m, coeff_var = self._make_model(X, y, k, warm_start)
        m.optimize()
        return coeff_var.X

    def _reduce(self, x, y):
        """
        Runs MIOSR either per dimension or jointly on all dimensions.

        Assumes an initial guess for coefficients and support are saved in
        ``self.coef_`` and ``self.ind_``.
        """
        if self.initial_guess is not None:
            self.coef_ = self.initial_guess

        n_samples, n_features = x.shape
        _, n_targets = y.shape

        if (
            self.target_sparsity is not None or self.constraint_lhs is not None
        ):  # Regress jointly
            coefs = self._regress(x, y, self.target_sparsity, self.initial_guess)
            # Remove nonzero terms due to numerical error
            non_active_ixs = np.argsort(np.abs(coefs))[: -int(self.target_sparsity)]
            coefs[non_active_ixs] = 0
            self.coef_ = coefs.reshape(n_targets, n_features)
            self.ind_ = (np.abs(self.coef_) > 1e-6).astype(int)
        else:  # Regress dimensionwise
            for i in range(n_targets):
                k = self.group_sparsity[i]
                warm_start = (
                    None if self.initial_guess is None else self.initial_guess[[i], :]
                )
                coef_i = self._regress(x, y[:, [i]], k, warm_start=warm_start)
                # Remove nonzero terms due to numerical error
                non_active_ixs = np.argsort(np.abs(coef_i))[: -int(k)]
                coef_i[non_active_ixs] = 0
                self.coef_[i, :] = coef_i
            self.ind_ = (np.abs(self.coef_) > 1e-6).astype(int)

    @property
    def complexity(self):
        check_is_fitted(self)
        return np.count_nonzero(self.coef_)


## Fitting Model

In [ ]:
coeffs = [15]
#coeffs = [5,10,15,20,25]
#coeffs = [10]
msqe = []
msqe2 = []
msqe3 = []

for n_coeff in coeffs:
    print("Model with",n_coeff, "coefficients")
    opt = MIOSR(target_sparsity = n_coeff, alpha=0.00000000000000001,
    constraint_lhs = constraintlhs, constraint_rhs = constraintrhs, gurobi_license=gurobi_license) #works with small alpha


    model = ps.SINDy(
        feature_library=weak_lin_lib, optimizer=opt, feature_names = ["U", "W", "P", "T"]
    )

    ## get the fitted udot integrals
    model.fit(X, t=time, unbias=False) #set unbias = False with MIOSR
    model.print(precision=8)
    msqe.append(np.sqrt(
        (
            np.sum((X_dot_train_integral - opt.Theta_ @ opt.coef_.T) ** 2, axis=0)
            / np.sum(X_dot_train_integral**2, axis=0)
        )
        / X_dot_train_integral.shape[0])) #calculate the msqe for each equation
    print("msqe")
    print(msqe[-1])


Model with 15 coefficients
(U)' = -0.03340485 P_2 + -0.02601786 P_1 + 0.21877644 UU_2 + -0.24019636 TP_2 + -0.43440500 UW_1
(W)' = -0.02601786 P_2 + 0.23201031 UU_2 + -0.23834060 UP_2 + -0.26241555 WW_1 + -0.20702971 TP_1
(P)' = 0.00000000
(T)' = -0.03160537 P_2 + 0.23751573 UU_2 + -0.23797974 TP_2 + -0.28089621 UU_1 + -0.20542198 UP_1
msqe
[0.04650275 0.04589116 0.05773503 0.04581172]


In [ ]:
opt = MIOSR(target_sparsity = None, group_sparsity=(3,4,0,2),
               alpha=0.00000000000000001, gurobi_license=gurobi_license)

model = ps.SINDy(
    feature_library=weak_lin_lib, optimizer=opt, feature_names = ["U", "W", "P", "T"]
)

print("Model with with group sparsity")
## get the fitted udot integrals
model.fit(X, t=time, unbias=False) #set unbias = False with MIOSR
model.print(precision=8)

msqe2.append(np.sqrt(
    (
        np.sum((X_dot_train_integral - opt.Theta_ @ opt.coef_.T) ** 2, axis=0)
        / np.sum(X_dot_train_integral**2, axis=0)
    )
    / X_dot_train_integral.shape[0])) #calculate the msqe for each equation
print("msqe2")
print(msqe2[-1])

Model with with group sparsity
(U)' = -0.02924635 P_2 + -0.22612343 WP_1 + -0.29870631 WT_1
(W)' = -0.03187794 P_2 + -0.26009051 WP_2 + -0.46137137 UU_1 + 0.24628704 TT_12
(P)' = 0.00000000
(T)' = -0.02264128 P_2 + -0.45581986 UP_1
msqe2
[0.04741998 0.04648299 0.05773503 0.04865107]


In [ ]:
print("Model with viscous terms")

opt = MIOSR(target_sparsity = None, group_sparsity=(5,6,0,4),
               alpha=0.00000000000000001, gurobi_license=gurobi_license)

model = ps.SINDy(
    feature_library=weak_lin_lib, optimizer=opt, feature_names = ["U", "W", "P", "T"]
)

## get the fitted udot integrals
model.fit(X, t=time, unbias=False) #set unbias = False with MIOSR
model.print(precision=8)

msqe3.append(np.sqrt(
    (
        np.sum((X_dot_train_integral - opt.Theta_ @ opt.coef_.T) ** 2, axis=0)
        / np.sum(X_dot_train_integral**2, axis=0)
    )
    / X_dot_train_integral.shape[0])) #calculate the msqe for each equation
print("msqe3")
print(msqe3[-1])


Model with viscous terms
(U)' = -0.03157813 P_2 + 0.23775464 UU_2 + -0.23821854 WP_2 + -0.28062726 UU_1 + -0.20571592 TP_1
(W)' = -0.03176051 P_2 + 0.22943447 UU_2 + -0.24590646 WP_2 + -0.28124540 UU_1 + -0.19298530 UP_1 + 0.03275456 PP_12
(P)' = 0.00000000
(T)' = -0.03187794 P_2 + -0.26009051 WP_2 + -0.46137137 UU_1 + 0.24628704 TT_12
msqe3
[0.04581172 0.04564351 0.05773503 0.04648299]
